# Create Entities

A short guide to adding Entities with Minimum Disambiguation Triples(MDTs) to the Golden decentralized knowledge graph using the godel SDK. Please see the [Golden Protocol FAQ Guide](https://www.notion.so/goldenhq/Golden-Protocol-FAQ-78ae2357b9af44aeaa655cb1b1966ee4) and the [Adding Structured Data Guide](https://www.notion.so/goldenhq/Adding-Structured-Data-Guide-ae657337bf4f4e54ae4402df083c76ac) to learn more about Golden, data submission, and rewards.

Note: Attribution and eligibility for testnet points on triple submissions will be assigned by the earliest timestamped transaction.

## Prerequisite

Godel [Authentication](https://docs.golden.xyz/api/godel-python-sdk/authentication)

This guide requires you install Godel's data-tools.

You can do this with `pip install godel` and comes pre-installed if using the godel docker image.

### 1. Connect to Golden Web3 API

Let's connect the python wrapper to the Golden GraphQL API.

Make sure you ran through the prerequisites for this guide and have learned how to authenticate and retrieve your JWT token in Godel.

In [ ]:
from godel import GoldenAPI

JWT_TOKEN = #YOUR_JWT_TOKEN_HERE
API_URL = "https://dapp.golden.xyz/graphql"
SANDBOX_URL = "https://sandbox.dapp.golden.xyz/graphql" # Use the sandbox API to test your submissions
goldapi = GoldenAPI(url=API_URL)
goldapi.set_jwt_token(jwt_token=JWT_TOKEN)

In [ ]:
import pandas as pd

# Test connection with search
search_results = goldapi.entity_search(name="Miles")
search_results_df = pd.DataFrame(search_results["data"]["entityByName"]["nodes"])
search_results_df.head()

### 2. Get Predicates and Templates

You can run the code below to get the list of accepted predicates into the knowledge graph, its data types, and submit templates for different kinds of entities.

In [ ]:
import pandas as pd

predicates = {}
for p in goldapi.predicates()["data"]["predicates"]["edges"]:
    p = p["node"]
    predicates[p["name"]] = {"id": p["id"], "objectType": p["objectType"]} 
predicates_df = pd.DataFrame(predicates).transpose()
predicates_df.head()

In [ ]:
templates = {}
for t in goldapi.templates()["data"]["templates"]["edges"]:
    t = t["node"]
    templates[t["entity"]["name"]] = {"id": t["id"], "entityId": t["entityId"], "entityDescription": t["entity"]["description"]} 
templates_df = pd.DataFrame(templates).transpose()
templates_df

### 3. Create Entity

#### Source Data

You will need to input the source data of the entity you would like to submit.

In [ ]:
name = "John Doe"
is_a = "0c4e6054-5fd8-48a8-817c-f6611278f755"  # Person Template Entity Id
ceo_of = "7338a7df-fe3a-45e6-8802-c3064d23dc66"  # Starbucks Entity ID
email_address = "john.doe@example.com"
citation_urls = ["https://golden.com/wiki/johndoe"]

#### Create Entity Input

We're going to use `CreateEntityInput`, which requires Minimum Disambiguated Triples (MDTs), to submit the new entity.

MDTs are the required triples you must submit along with the entity you want to create. This helps us with disambiguation, deduplication, and arbitrary entity submissions with zero triples.

First, view the `StatementInputRecordInput` object schema.

In [ ]:
from godel.schema import CreateEntityInput, StatementInputRecordInput

print(repr(CreateEntityInput))
StatementInputRecordInput

Create your triples with `StatementInputRecordInput`.

You'll notice that the "CEO of" statement is commented out. Without this, even if you have the email address statement, you will not be able to submit the entity since it does not fulfill the MDTs required. We are using pandas here to make working with data easier, but it is not a requirement.

Remove the comment-out of the "CEO of" statement to succesfully submit the entity.

In [ ]:
# Create triples inputs
statements = []

# Add Template
statements.append(
    StatementInputRecordInput(
        predicate_id=predicates["Is a"]["id"],
        object_entity_id=is_a,
        citation_urls=citation_urls,
        qualifiers=[],
    )
)

# Add Name
statements.append(
    StatementInputRecordInput(
        predicate_id=predicates["Name"]["id"],
        object_value=name,
        citation_urls=citation_urls,
        qualifiers=[],
    )
)

# Add Email Address
statements.append(
    StatementInputRecordInput(
        predicate_id=predicates["Email Address"]["id"],
        object_value=email_address,
        citation_urls=citation_urls,
        qualifiers=[],
    )
)

# Add CEO of
# statements.append(
#     StatementInputRecordInput(
#         predicate_id=predicates["CEO of"]["id"],
#         object_entity_id=ceo_of,
#         citation_urls=citation_urls,
#         qualifiers=[],
#     )
# )

Now that you have the statements, you can create your `CreateEntityInput`.

In [ ]:
# Create Entity Input
create_entity_input = CreateEntityInput(
    statements=statements
)
create_entity_input.__to_json_value__()

### WARNING: Running the code below may charge gas fees and stake testnet points with your wallet. You may lose testnet points by submitting incorrect data.

### Submit entity data to protocol

In [ ]:
data = goldapi.create_entity(create_entity_input=create_entity_input)
data

In [ ]:
# Below is a dataframe of your new entity once submitted with the required MDTs. 
created_data_df = pd.DataFrame([data["data"]["createEntity"]["entity"]])
created_data_df

### 5. View Data on dApp

In [ ]:
created_entity_id = data["data"]["createEntity"]["entity"]["id"]
created_entity_id
link = f"https://dapp.golden.xyz/entity/{created_entity_id}"
link